In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from jellyfish.core import Client
from jellyfish.history_loader import load_candles_history
from jellyfish import utils
from datetime import datetime, timedelta

utils.disable_warnings()

pair = 'btcusdt'
end_dt = datetime(year=2021, month=9, day=25)
frame = load_candles_history(Client(), pair, end_dt=end_dt, interval='1m', candles_num=5000 * 4 * 2)
end_dt

  0%|          | 0/40 [00:00<?, ?it/s]

datetime.datetime(2021, 9, 25, 0, 0)

In [33]:
from jellyfish import indicator
import numpy as np

returns = frame.Close / frame.Open
hurst = indicator.hurst(returns, kind=indicator.HURST_CHANGE, simplified=False)

  0%|          | 0/39941 [00:00<?, ?it/s]

In [34]:
th = hurst[100:].mean()
th_eps = hurst[100:].std() * 1e-3
th_eps = hurst[100:].std() * 3.5e-1

frame['Persistent'] = (hurst > th + th_eps).astype(np.int32)
frame['Reversal'] = (hurst < th - th_eps).astype(np.int32)
frame['RandomWalk'] = (np.abs(hurst - th) < th_eps).astype(np.int32)
frame['i_Hurst'] = hurst

In [35]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG
from jellyfish.core import SmaCross, Backtest

condition = lambda ohlc: ohlc.Persistent.max() + ohlc.Reversal.max() > 1
frame1 = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')

  0%|          | 0/40040 [00:00<?, ?it/s]

In [36]:
bt = Backtest(frame1, SmaCross, trade_on_close=True)
stats = bt.run(
    n1=5,
    n2=20
)
bt.plot(open_browser=True)
stats

Start                     2021-08-28 11:04:00
End                       2021-09-25 03:00:00
Duration                     27 days 15:56:00
Exposure Time [%]                    97.38339
Equity Final [$]               1075653.904296
Equity Peak [$]                1160011.590936
Return [%]                            7.56539
Buy & Hold Return [%]              -12.595993
Return (Ann.) [%]                  150.401754
Volatility (Ann.) [%]              162.430618
Sharpe Ratio                         1.873449
Calmar Ratio                        11.917251
Max. Drawdown [%]                   -9.667991
Avg. Drawdown [%]                   -2.464498
Max. Drawdown Duration        7 days 14:22:00
Avg. Drawdown Duration        1 days 04:59:00
# Trades                                   50
Win Rate [%]                             32.0
Best Trade [%]                       8.763043
Worst Trade [%]                     -5.343425
Avg. Trade [%]                        0.14668
Max. Trade Duration           2 da

In [17]:
utils.plot_ohlc(frame, open_browser=True);